In [11]:
def extract_actions(single_history):
    """
    Takes a single string of history (no /) and 
    returns a list of actions in a form such as
    ['c', '1r', '10r', 'f']
    """
    actions = []
    current_action = ''
    for item in single_history:
        if item in ['a', 'c', 'f']:
            actions.append(item)
            current_action = ''
        elif item.isdigit():
            current_action += item
        elif item == 'r':
            actions.append(current_action + item)
            current_action = ''
    return actions
            

def player_money_bet(action_history):
    """
    Returns the amount of money p1 and p2 have bet
    (not including the antee) with the format of p1, p2.
    Assumes that player 1 always moves first for each betting round.
    """
    p1_commited = 0
    p2_commited = 0
    for history in action_history:
        p1_temp = 0
        p2_temp = 0
        for key, action in enumerate(extract_actions(history)):
            if key % 2 == 0:
                if action == 'c':
                    p1_temp = p2_temp
                elif 'r' in action:
                    p1_temp += int(action.replace('r', ''))
                elif action == 'a':
                    p1_temp += 1
            else:
                if action == 'c':
                    p2_temp = p1_temp
                elif 'r' in action:
                    p2_temp += int(action.replace('r', ''))
                elif action == 'a':
                    p2_temp += 1
        p1_commited += p1_temp
        p2_commited += p2_temp
    
    return p1_commited, p2_commited

def valid_actions(infoset_key, max_bet):
    """
    Returns a list of valid actions based off the tree history.
    """
    player_id = infoset_key.split(";")[0]
    action_history = infoset_key.split(";")[-1].split('/')

    p1_commited, p2_commited = player_money_bet(action_history)
    if p1_commited > max_bet or p2_commited > max_bet:
        error_msg = f"P1 or P2 have bet too much money! p1_commited: {p1_commited} and p2_commited: {p2_commited}"
        error_msg += f" with a max bet of {max_bet}"
        raise ValueError(error_msg)

    print(p1_commited, p2_commited)
    actions = ['c']
    if player_id == "P1":
        if p2_commited > p1_commited: # Being raised against
            actions = ['f'] + actions
        for bet_amount in range(1, max_bet + 1 - p1_commited):
            actions.append(str(bet_amount) + 'r')

    if player_id == "P2":
        if p1_commited > p2_commited: # Being raised against
            actions = ['f'] + actions
        for bet_amount in range(1, max_bet + 1 - p2_commited):
            actions.append(str(bet_amount) + 'r')

    return actions

class Card:
    SUIT_TO_STRING = {
        1: "s",
        2: "h",
        3: "d",
        4: "c"
    }
    
    RANK_TO_STRING = {
        2: "2",
        3: "3",
        4: "4",
        5: "5",
        6: "6",
        7: "7",
        8: "8",
        9: "9",
        10: "T",
        11: "J",
        12: "Q",
        13: "K",
        14: "A"
    }

    RANK_JACK = 11
    RANK_QUEEN = 12
    RANK_KING = 13
    RANK_ACE = 14
    
    STRING_TO_SUIT = dict([(v, k) for k, v in SUIT_TO_STRING.items()])
    STRING_TO_RANK = dict([(v, k) for k, v in RANK_TO_STRING.items()])

    def __init__(self, rank, suit):
        """
        Create a card. Rank is 2-14, representing 2 through Ace,
        while suit is 1-4 representing spades, hearts, diamonds, clubs
        """
        self.rank = rank
        self.suit = suit

    def __repr__(self):
        return "(%s, %s)" % (self.rank, self.suit)
    
    def pretty_repr(self):
        return "%s%s" % (self.RANK_TO_STRING[self.rank], self.SUIT_TO_STRING[self.suit])
    
    def __eq__(self, other):
        return (isinstance(other, self.__class__) and self.rank == other.rank and self.suit == other.suit)
    
    def __hash__(self):
        return hash((self.rank, self.suit))

def evaluate_winner(cards_1, cards_2, community_cards, history):
    """
    This function returns 1, 2, or -1 for player winning,
    opponent winning, or a tie. 
    
    For this modified version of Kuhn
    poker (J, Q, K, A), it just requires comparing the cards of each
    player and the winner is the one with the higher card. There are
    no ties.
    """
    print(",".join([x.pretty_repr() for x in cards_1]))
    print(",".join([x.pretty_repr() for x in cards_2]))
    print(",".join([x.pretty_repr() for x in community_cards]))
    
    # Check for folding
    final_history = history[-1]
    if len(final_history) > 0 and final_history[-1] == 'f':
        if len(final_history) % 2 == 0:
            return 1
        else:
            return 2

    # Assuming no one folded, evaluate based on high card
    p1_best_card = max([int(card.rank) for card in cards_1])
    p2_best_card = max([int(card.rank) for card in cards_2])
    p1_worst_card = min([int(card.rank) for card in cards_1])
    p2_worst_card = min([int(card.rank) for card in cards_2])

    # Check for 2 pair
    if set([x.rank for x in cards_1]).intersection(set([x.rank for x in community_cards])):
        return 1
    elif set([x.rank for x in cards_2]).intersection(set([x.rank for x in community_cards])):
        return 2

    # Check for high card
    if p1_best_card > p2_best_card:
        return 1
    if p2_best_card > p1_best_card:
        return 2
    if p1_best_card == p2_best_card:
        if p1_worst_card > p2_worst_card:
            return 1
        if p2_worst_card > p1_worst_card:
            return 2
        return -1

print(valid_actions("P1;;;aa/cc/", 4))
#print(evaluate_winner((Card(3,1), Card(3,2)), (Card(2,1), Card(2,2)), [(Card(4,2))], ['rRc']))

1 1
['c', '1r', '2r', '3r']


In [3]:
def is_terminal_node(infoset_key, betting_rounds, max_bet):
    action_history = infoset_key.split(";")[-1].split('/')
    if len(action_history) > 1:
        
        if len(action_history) < betting_rounds + 1: # +1 is for antee round
            p1_commited, p2_commited = player_money_bet(action_history)
            if p1_commited == max_bet and p2_commited == max_bet:
                return True # Players have all in before the final round
        
        final_history = action_history[-1] # Only look at the latest round
        if len(final_history) > 0 and 'f' == final_history[-1]:
            return True # A player folded
        if len(final_history) > 1:
            final_moves = final_history[-2:] # Both players have taken at least one action
            if final_moves in ['cR', 'Rc']:
                return True
            if len(action_history) == betting_rounds + 1:
                if final_moves in ['cc', 'rc', 'cr']:
                    return True
    return False

print(is_terminal_node('P1;2;0;aa/cc/cc', 2, 3))

False
